In [ ]:
from collections import deque
from collections import defaultdict

In [369]:
input_file = "input.txt"

In [370]:
def parse_name(name, dests):
    if name == 'broadcaster':
        return {'name':'broadcaster', 'type':'broadcaster', 'params':{}, 'dests':dests}
    if name[0] == '%':
        return {'name':name[1:], 'type':'flip-flop', 'params':{'state':False}, 'dests':dests}
    if name[0] == '&':
        return {'name':name[1:], 'type':'conjunction', 'params':{'memory':{}}, 'dests':dests}

In [ ]:
def processFactory(m):
    def process(source, pulse):
        if modules[m]['type'] == 'output':
            return

        if modules[m]['type'] == 'broadcaster':
            for dest in modules[m]['dests']:
                    fifo.append((m, pulse, dest))

        if modules[m]['type'] == 'flip-flop':
            if(pulse):
                return
            modules[m]['params']['state'] = not modules[m]['params']['state']
            for dest in modules[m]['dests']:
                fifo.append((m, modules[m]['params']['state'], dest))

        if modules[m]['type'] == 'conjunction':
            modules[m]['params']['memory'][source] = pulse
            if all(modules[m]['params']['memory'][k] for k in modules[m]['params']['memory']):
                #sends low
                for dest in modules[m]['dests']:
                    fifo.append((m, False, dest))
            else:
                #sends high
                for dest in modules[m]['dests']:
                    fifo.append((m, True, dest))

    return process

In [ ]:
lines = open(input_file, 'r').readlines()
lines = [line.split(' -> ') for line in lines]
lines = [(parse_name(name,dests.strip().split(', '))) for name, dests in lines]
modules = {}

for line in lines:
    modules[line['name']]=line

output_modules = []
for m in modules:
    for d in modules[m]['dests']:
        if d not in modules:
            output_modules.append(d)

for out_m in output_modules:
    modules[out_m] = {'name':out_m, 'type':'output', 'dests':[]}

for m in modules:
    for d in modules[m]['dests']:
        if modules[d]['type'] == 'conjunction':
            modules[d]['params']['memory'][m] = False

for k in modules:
    p = processFactory(k)
    modules[k]['process'] = p

In [ ]:
lows = 0
highs = 0
for _ in range(0,1000):
    fifo = deque([('button', False,'broadcaster')])
    while len(fifo) > 0:
        source, pulse, dest = fifo.popleft()
        if pulse:
            highs += 1
        else:
            lows += 1
        #print(source, '-[', pulse ,']->', dest)
        modules[dest]['process'](source, pulse)
part1 = lows*highs
part1

# part 2 - does not scale

In [ ]:
part2 = 0
conjuction_cycles = {}
for _ in range(0,5000):
    part2 += 1
    low_sent_to_rx = False
    fifo = deque([('button', False,'broadcaster')])
    while len(fifo) > 0:
        source, pulse, dest = fifo.popleft()
        if dest == 'rx' and not pulse:
            low_sent_to_rx = True
            break
        #print(source, '-[', pulse ,']->', dest)
        modules[dest]['process'](source, pulse)
    if low_sent_to_rx:
        break

# part 2 - conjunction of conjunction cycles

In [389]:
def processFactory(m):
    def process(source, pulse, i):
        if modules[m]['type'] == 'output':
            return

        if modules[m]['type'] == 'broadcaster':
            for dest in modules[m]['dests']:
                    fifo.append((m, pulse, dest))

        if modules[m]['type'] == 'flip-flop':
            if(pulse):
                return
            modules[m]['params']['state'] = not modules[m]['params']['state']
            for dest in modules[m]['dests']:
                fifo.append((m, modules[m]['params']['state'], dest))

        if modules[m]['type'] == 'conjunction':
            modules[m]['params']['memory'][source] = pulse
            if all(modules[m]['params']['memory'][k] for k in modules[m]['params']['memory']):
                #sends low
                for dest in modules[m]['dests']:
                    fifo.append((m, False, dest))
                    # here we compute the cycles
                    if m not in conjuction_cycles:
                        conjuction_cycles[m] = i
            else:
                #sends high
                for dest in modules[m]['dests']:
                    fifo.append((m, True, dest))

    return process

In [390]:
lines = open(input_file, 'r').readlines()
lines = [line.split(' -> ') for line in lines]
lines = [(parse_name(name,dests.strip().split(', '))) for name, dests in lines]
modules = {}

for line in lines:
    modules[line['name']]=line

output_modules = []
for m in modules:
    for d in modules[m]['dests']:
        if d not in modules:
            output_modules.append(d)

for out_m in output_modules:
    modules[out_m] = {'name':out_m, 'type':'output', 'dests':[]}

for m in modules:
    for d in modules[m]['dests']:
        if modules[d]['type'] == 'conjunction':
            modules[d]['params']['memory'][m] = False

for k in modules:
    p = processFactory(k)
    modules[k]['process'] = p

In [400]:
from functools import reduce

lows = 0
highs = 0
conjuction_cycles = {}
for i in range(1,10000):
    fifo = deque([('button', False,'broadcaster')])
    while len(fifo) > 0:
        source, pulse, dest = fifo.popleft()
        #print(source, '-[', pulse ,']->', dest)
        modules[dest]['process'](source, pulse,i)

part2 = reduce(lambda a,b: a*b, conjuction_cycles.values())
part2

29728720800